In [2]:
import pandas as pd 
import keras
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder,OneHotEncoder
from sklearn.pipeline import Pipeline
from scikeras.wrappers import KerasClassifier
import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import pickle


In [3]:
data = pd.read_csv("Churn_Modelling.csv")

#drop Irrelevant columns
data= data.drop(["RowNumber","CustomerId","Surname"],axis=1)
data.head()

label_encoder_gender = LabelEncoder()
data["Gender"] = label_encoder_gender.fit_transform(data["Gender"])

Onehot_encoder_geo = OneHotEncoder()
geo_encoder = Onehot_encoder_geo.fit_transform(data[["Geography"]])
Onehot_encoder_geo.get_feature_names_out(["Geography"])
geo_encoded_df = pd.DataFrame(geo_encoder.toarray(),columns=Onehot_encoder_geo.get_feature_names_out(["Geography"]))
data = pd.concat([data.drop(["Geography"],axis=1),geo_encoded_df],axis=1)
X= data.drop(["Exited"], axis=1)
y=data['Exited']

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=43)

scalar = StandardScaler()
X_train = scalar.fit_transform(X_train)
X_test = scalar.transform(X_test)

In [10]:
##Save Encoders and Scalers
with open("label_encoder_gender.pkl",'wb') as file:
    pickle.dump(label_encoder_gender,file)
with open("Onehot_encoder_geo.pkl",'wb') as file:
    pickle.dump(Onehot_encoder_geo,file)

In [22]:
def model_exp(neurons=32, layer=1):
    model = Sequential()
    model.add(Dense(neurons, activation='relu',input_shape=(X_train.shape[1],)))
    for _ in range(layer-1):
        model.add(Dense(neurons, activation='relu'))

    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

    return model 


In [26]:
## Create Keras Classifier 

model = KerasClassifier(neurons=32, layer=1,build_fn=model_exp, verbose=0)

In [27]:
param_grid = {
"neurons":[16,32,64,128],
"layer":[1,2],
"epochs":[50,100]
}

In [28]:
grid = GridSearchCV(estimator=model, param_grid=param_grid,n_jobs=-1,cv=3)
grid_results = grid.fit(X_train,y_train)    

print(f"Best {grid_results.best_score_} using {grid_results.best_params_}")

c:\Users\mrafi\Documents\ANN\ANN\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\mrafi\Documents\ANN\ANN\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Best 0.8586252768666482 using {'epochs': 50, 'layer': 2, 'neurons': 16}
